In [1]:
import numpy as np 

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVR, SVR, LinearSVR
from sklearn.neighbors import KNeighborsClassifier
# different regressor test
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet

import pandas as pd # process txt
from scipy.io import loadmat
from PIL import Image

from collections import defaultdict

In [2]:
folder = "G:/dataset/cub/CUB_200_2011/"
standard_path = "G:/dataset/standard_split/CUB/"
proposed_path = "G:/dataset/proposed_split/CUB/"

cls_to_idx = {}
with open(folder + "classes.txt", "r", encoding='utf-8') as f:
     for row in f.readlines():
         row = row.rstrip()
         idx, name = row.split()
         cls_to_idx[name] = int(idx) - 1 # custom will -1

sstrain, sstest = [], []
pstrain, pstest = [], []

# Standard Split 
with open(standard_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstrain.append(cls_to_idx[row])

with open(standard_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstest.append(cls_to_idx[row])

print("standard_split:", len(sstrain), len(sstest))
# transform List(str) -> List(int)

# Proposed Split
with open(proposed_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstrain.append(cls_to_idx[row])

with open(proposed_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstest.append(cls_to_idx[row])

print("proposed_split:", len(pstrain), len(pstest))


# Random Train & Test Class Split

train_class, test_class = [], [] # (5994, 5794)
X_class = list(range(200)) # label start from 0, you can adjust it

#train_class, test_class = train_test_split(X_class, test_size=0.2)
#train_class, test_class = sstrain, sstest

train_class, test_class = pstrain, pstest

standard_split: 150 50
proposed_split: 150 50


In [3]:
print(*test_class, sep = ", ")

42, 110, 22, 97, 54, 129, 138, 122, 155, 123, 199, 71, 172, 27, 118, 164, 102, 179, 76, 11, 44, 189, 190, 137, 156, 51, 32, 163, 30, 142, 93, 69, 96, 90, 103, 126, 160, 48, 168, 147, 112, 86, 162, 135, 187, 83, 25, 3, 131, 167


In [4]:
test_class[:10]

[42, 110, 22, 97, 54, 129, 138, 122, 155, 123]

In [5]:
class cubRead:
    def __init__(self, p, train_split):
        """
        p: 数据集存放路径
        train_split: 给出训练集
        """
        X_class = list(range(200)) # custom: range(200)
        train_class = train_split
        test_class = list(filter(lambda i: i not in train_class, X_class))
        self.path = p
        # labels
        yp = self.path + "custom-cub-labels.txt"
        y = np.loadtxt(yp, delimiter=" ", encoding='utf-8')
        # visual features 2048 dimensions
        xp = self.path + "custom-cub-features.txt"
        x = np.loadtxt(xp, delimiter=" ", encoding='utf-8')
        i1 = np.isin(y, train_class)
        i2 = np.isin(y, test_class)
        self.X_train, self.X_test = x[i1], x[i2]
        self.y_train, self.y_test = y[i1], y[i2]
    def train_data(self):
        return self.X_train, self.y_train
    def test_data(self):
        return self.X_test, self.y_test
    def test(self): # 11788 all, add them to test
        print(self.X_train.shape, self.y_train.shape)
        print(self.X_test.shape, self.y_test.shape)

cubReader = cubRead(folder, train_class)
cubReader.test() # 8855 + 2933 = 11788
# their cub data, 8823 + 2965, 有点问题

(8821, 2048) (8821,)
(2967, 2048) (2967,)


In [6]:
X_train, y_train = cubReader.train_data()
X_test, y_test = cubReader.test_data()
#X_train.shape, y_train.shape, X_test.shape, y_test.shape
## Hyper Parameters

# dimension of PCA

pca_d = 500

exemPCA = decomposition.PCA(n_components=pca_d)
exemPCA.fit(X_train)
X_train = exemPCA.transform(X_train)
X_test = exemPCA.transform(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8821, 500), (8821,), (2967, 500), (2967,))

In [7]:
X_train

array([[ 1.28562331e+01, -3.07478366e+00, -2.56507953e+00, ...,
         2.24887072e-01, -5.02215694e-02,  2.07814415e-02],
       [ 1.32626356e+01, -2.49965146e+00, -1.92945262e+00, ...,
        -1.42076074e-01, -2.16375865e-02,  2.32117040e-01],
       [ 1.36109539e+01, -1.45723334e+00, -1.86439895e+00, ...,
        -3.48232371e-02,  1.99272638e-01, -1.01119538e-01],
       ...,
       [-2.30579039e+00, -2.89017333e+00,  9.98339794e-01, ...,
        -6.19876985e-02, -2.46070661e-03,  5.60379742e-02],
       [-2.50973656e-01, -2.99399764e+00,  2.82779538e+00, ...,
        -1.34863796e-01, -2.36190579e-02,  1.31164547e-01],
       [ 4.22318316e-01, -5.69984702e+00,  4.40918279e+00, ...,
        -1.47719562e-02,  3.91515213e-02, -8.82176305e-02]])

In [8]:
# group up PCA projections

exem_train_group = defaultdict(list)

for c in train_class:
    exem_train_group[c] = []
        
for x, y in zip(X_train, y_train):
    exem_train_group[y].append(x)

# Average
exem_train = {}
std_train = {}
# standard deviation

k = 0

for item in exem_train_group.items():
    y, ary = item
    exem_train[y] = np.mean(ary, axis=0) # Key Sentence
    std_train[y] = np.std(ary, axis=0)

del exem_train_group


# class to index, the same
# 如果是lasso，不能标准化
# semantic embedding of CUB, 200 birds, 312 attributes
se = folder + "attributes/class_attribute_labels_continuous.txt"
semat = np.loadtxt(se, delimiter=" ", encoding='utf-8')

#semat = normalize(semat, norm='l2')

a_c_train, a_c_test = semat[train_class], semat[test_class]
v_c_train = [exem_train[i] for i in train_class] # dict uses classes to index
sd_train = [std_train[i] for i in train_class]
sd_train = np.mean(sd_train, axis=0)

In [9]:
np.array(a_c_train).shape, np.array(v_c_train).shape, np.array(sd_train).shape

((150, 312), (150, 500), (500,))

In [43]:
regress_group = []
k = 0
for j in range(pca_d):
    X = a_c_train # 
    y = [vc[j] for vc in v_c_train]
    regressor = Lasso(alpha=1)
    #regressor = NuSVR(C=2)
    regressor.fit(X, y)
    regress_group.append(regressor)

v_c_test = np.zeros((len(test_class), pca_d)) # 提前定义好exemplar矩阵
# 对每一个维度进行预测
for j in range(pca_d):
    v_c_test[:, j] =  regress_group[j].predict(a_c_test) # 10 dimension , assign to column

print(v_c_test.shape, len(v_c_train), len(v_c_train[0]))

exem_X, exem_y = [], []

for i, c in enumerate(test_class):
    exem_X.append(v_c_test[i])
    exem_y.append(c)



neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(exem_X, exem_y)
print("1NN:{} ".format(neigh.score(X_test, y_test)))

sneigh = KNeighborsClassifier(n_neighbors=1, metric='seuclidean', 
                    metric_params={'V':sd_train})
sneigh.fit(exem_X, exem_y)
print("1NNs:{}".format(sneigh.score(X_test, y_test)))

(50, 500) 150 500
1NN:0.47724974721941354 
1NNs:0.5032018874283788


In [44]:
regress_group[4].predict(a_c_test), regress_group[4].predict(a_c_test).shape, regress_group[5].predict(a_c_test)

(array([ 1.86710918,  1.79679195, -0.52144775, -2.05599093, -0.2326713 ,
         1.57456961, -0.46563282, -0.99361592,  1.68093002,  0.32696533,
         0.83398897,  1.2802974 ,  0.3472438 , -0.15449661,  1.76555659,
         1.25386149,  0.93204596,  0.90415291,  2.0464996 , -0.76837355,
         2.36691703, -4.69707948,  0.12536649, -1.14341364,  0.89576475,
        -0.14912788,  2.01963394, -0.63166289,  1.85415264,  2.21158489,
         0.70673272, -1.53107535, -1.85956187,  1.27224217,  0.18085042,
        -0.18610892,  0.9541327 , -2.53800272,  0.06820689,  0.46773325,
        -0.22879981,  0.10627698, -1.23894894, -0.62223353, -1.6770004 ,
         2.05332239, -2.64863689, -2.23173553,  1.17372781,  0.54764598]),
 (50,),
 array([-0.03663805, -0.78066415, -0.93461369,  1.61118045,  0.10878492,
        -0.94441111, -2.58347483, -0.31397552,  0.43414766, -0.13552903,
         1.0532155 , -0.91780978,  0.57914565,  0.60233728, -0.48516376,
        -0.23659848, -0.70632464,  1.0620

In [45]:
## Save Exemplars and visual features of test class
#save_path = "./cub_exem/"
#np.save(save_path + "exem_test.npy", np.array(exem_X))
#np.save(save_path + "X_test.npy", X_test)
#np.savetxt(save_path + "y_test.txt", np.array(y_test, dtype=int), fmt='%s', encoding='utf-8')

In [46]:
# accuracy = neigh.score(X_test, y_test) deprecated

# 4.7 Random Split
# ZSL 0.6281342966425839
# GZSL 0.23799405014874628

# 4.8 Random Split
# ZSL 0.6365180467091295

# 4.12 Standard Split
# ZSL 0.5744971019434026
# GZSL 0.18581657006478008

# 4.16 Proposed Split
# ZSL 0.5466801482979441
# MLP variant 0.5284799460734749
#